<a href="https://colab.research.google.com/github/Amir-Mohseni/Fine-tuning-Llama-3.1-8b/blob/main/Llama_3_1_Fine_tune_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers datasets trl accelerate bitsandbytes openai peft tensorboard

      Successfully uninstalled transformers-4.42.4
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.17.1 which is incompatible.


In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the base model and the adapter model
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
adapter_model = "AmirMohseni/Llama-3.1-8B-Instruct-Persian-finetuned-sft"

# Load the base model and apply the adapter model using PEFT
model = AutoModelForCausalLM.from_pretrained(base_model, device_map={"": 0})
model = PeftModel.from_pretrained(model, adapter_model)

# Check if CUDA is available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
# Add a new pad token if necessary
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Adding a distinct pad token

# Example usage
input_text = "چطوری میتونم به اطلاعات درباره ی سهام شرکت های آمریکایی دست پیدا کنم؟"

# Tokenize the input and get both input IDs and attention mask
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Generate text
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=512, pad_token_id=tokenizer.pad_token_id)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

چطوری میتونم به اطلاعات درباره ی سهام شرکت های آمریکایی دست پیدا کنم؟ (پاسخ)
برخی از منبع های مهمی که میتونید برای دسترسی به اطلاعات درباره ی سهام شرکت های آمریکایی استفاده کنید عبارتند از:
1. وبسایت رسمی شرکت: اکثر شرکت ها اطلاعات مربوط به سهام خود را بر روی وبسایت رسمی خود منتشر می کنند. این شامل اطلاعات درباره ی تاریخچه ی شرکت، محصولات و خدمات، استراتژی های تجاری، اطلاعات مالی، و جزئیات مربوط به سهام و سرمایه گذاری است.
2. وبسایت های سهام و سرمایه گذاری: وبسایت های مانند Yahoo Finance، Google Finance، و Morningstar، اطلاعات کاملی درباره ی سهام شرکت های آمریکایی را در اختیار دارا هستند. این وبسایت ها اطلاعات درباره ی قیمت های سهام، حجم معاملات، اطلاعات مالی، و رتبه بندی شرکت ها را ارائه می دهند.
3. وبسایت های تجارت و اقتصاد: وبسایت های مانند Bloomberg، CNBC، و Forbes، اخبار و گزارش های مرتبط با شرکت ها و بازار سهام را منتشر می کنند.
4. وبسایت های داده های مالی: وبسایت هایی مانند EDGAR (SEC)، که توسط سازمان بورس و اوراق بهادار آمریکا (SEC) اداره می شود، اطلاعات مالی و گزارش های شرکت ه

In [ ]:
# Save the combined model and tokenizer locally
model.save_pretrained("./combined-model")
tokenizer.save_pretrained("./combined-tokenizer")

('./combined-tokenizer/tokenizer_config.json',
 './combined-tokenizer/special_tokens_map.json',
 './combined-tokenizer/tokenizer.json')

In [ ]:
from huggingface_hub import login

# Step 1: Login to Hugging Face Hub (Only needed if not already logged in)
login()

# Step 2: Push the combined model and tokenizer to Hugging Face Hub
combined_model_repo = "AmirMohseni/Llama-3.1-8B-Instruct-Persian-finetuned-sft"

model.push_to_hub(combined_model_repo)
tokenizer.push_to_hub(combined_model_repo)

README.md:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AmirMohseni/Llama-3.1-8B-Instruct-Persian-finetuned-sft/commit/7312d64a1991bedf13d8bc592359330f2d528a3c', commit_message='Upload tokenizer', commit_description='', oid='7312d64a1991bedf13d8bc592359330f2d528a3c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import pipeline

# Specify the model
model_name = "AmirMohseni/Llama-3.1-8B-Instruct-Persian-finetuned-sft"

# Use the pipeline as a high-level helper
pipe = pipeline(
    "text-generation",
    model=model_name,
    trust_remote_code=True,
    device_map={"": 0},
    max_length=512,  # Adjust this value as needed
    temperature=0.2,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Prepare the message
messages = [
    {"role": "user", "content": "چطوری باید از دهان شویه استفاده کنم؟"},
]

# Generate a response
response = pipe(messages[0]["content"])
print(response[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


چطوری باید از دهان شویه استفاده کنم؟ برای استفاده از شویه دهانی، ابتدا شویه را با آب گرم مخلوط کنید. سپس با یک قاشق چایخوری از مخلوط شویه و آب به داخل دهان خود بریزید. بعد از 30 ثانیه، شویه را بلعید. این کار را 3 تا 4 بار در روز تکرار کنید.
شویه دهان چه فوایدی دارد؟
شویه دهان می‌تواند به کاهش عفونت‌های دهان و دندان کمک کند. همچنین می‌تواند به کاهش التهاب و درد در دهان و دندان کمک کند. شویه دهان می‌تواند به پاکسازی دهان و دندان کمک کند و به جلوگیری از تشکیل پلاک دندان کمک کند.
شویه دهان چه زمانی استفاده شود؟
شویه دهان می‌تواند در هر زمان استفاده شود، اما بهترین زمان برای استفاده از شویه دهان بعد از غذا و قبل از خواب است. همچنین می‌توانید شویه دهان را قبل از انجام هر کاری که نیاز به دهان تمیز دارید، استفاده کنید.
شویه دهان چه کسانی باید استفاده کنند؟
شویه دهان برای هر کسی که به دهان و دندانش اهمیت می‌دهد، مناسب است. اما اگر شما دچار بیماری‌های دهان و دندان هستید، قبل از استفاده از شویه دهان، با پزشک خود مشورت کنید.
شویه دهان چه کسانی نباید استفاده کنند؟
شویه دهان برای افراد زیر 6 سال منا